### HW3 CSE158 - 
# Purchase and Category Predictor
By Nicolas Carmont Zaragoza (A15677088)

1.) TASK: Build negative sample set by randomly sampling 100k non-purchased user/item pairs (as so far we only have postive sample data). Then combine the 100k negative sample with the 100k positive sample to create the validation set and report its accuracy on the baseline.

In [1]:
# Load in file and item/pair data
import gzip
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)
    
### Rating baseline: compute averages for each user, or return the global average if we've never seen the user before

allRatings = []
userRatings = defaultdict(list)
allData = list()

for data in readGz("train.json.gz"):
    allData.append(data)
    
trnData = allData[:100000]
validationData = allData[100000:]

import random

users1 = list(set(a['reviewerID'] for a in allData))  # convert to set to remove duplicate users
items1 = list(set(a['itemID'] for a in allData))

userItemPairs = set((a['reviewerID'],a['itemID']) for a in allData)

      
validationSet2 = set()
while(len(validationSet2)!=100000):
    userRand = random.choice(users1)
    itemRand = random.choice(items1)
    
    if(((userRand,itemRand) not in userItemPairs) & ((userRand,itemRand) not in validationSet2)):
        validationSet2.add((userRand,itemRand)) # check for duplicates                  
        validationData.append({'reviewerID':userRand,'itemID':itemRand})
        

In [2]:
# Run baseline
businessCount = defaultdict(int)
totalPurchases = 0

# for l in readGz("train.json.gz"):
for l in trnData:
    user,business = l['reviewerID'],l['itemID']
    businessCount[business] += 1
    totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/2:  
        break
# print(len(return1))

TP = 0
TN = 0
FP = 0
FN = 0

# predictions = open("predictions_Purchase.txt", 'w')
for j in range(0,len(validationData)):
    u,i = validationData[j]['reviewerID'],validationData[j]['itemID']
    if ((i in return1) & (j<100000)):
        TP+=1
    elif ((i in return1) & (j>=100000)):
        FP+=1
    elif((i not in return1) & (j>=100000)):
        TN+=1
    elif((i not in return1) & (j<100000)):
        FN+=1
        
print("Performance of baseline Q1:")      
print("TP: " + str(TP))
print("TN: " +str(TN))
print("FN: " +str(FN))
print("FP: " + str(FP))
total = TP+FP+FN+TN
print("Total: "+ str(total))
accuracy = (TP+TN)/total
print("Accuracy: " + str(accuracy))


Performance of baseline Q1:
TP: 44482
TN: 81392
FN: 55518
FP: 18608
Total: 200000
Accuracy: 0.62937


2.) TASK:The current purchase prediction baseline return True if an item is within the top 50th percentile of popularity. Is this threshold the best?

In [3]:
# Run Baseline with different threshold values
businessCount = defaultdict(int)
totalPurchases = 0

# for l in readGz("train.json.gz"):
for l in trnData:
    user,business = l['reviewerID'],l['itemID']
    businessCount[business] += 1
    totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/0.5:  # 0.1, 0.5, 0.8, 2,3, 10, 200, 10000 change values
        break

TP = 0
TN = 0
FP = 0
FN = 0

for j in range(0,len(validationData)):
    u,i = validationData[j]['reviewerID'],validationData[j]['itemID']
    if ((i in return1) & (j<100000)):
        TP+=1
    elif ((i in return1) & (j>=100000)):
        FP+=1
    elif((i not in return1) & (j>=100000)):
        TN+=1
    elif((i not in return1) & (j<100000)):
        FN+=1

print("Performance of baseline Q2:") 
print("TP: " + str(TP))
print("TN: " +str(TN))
print("FN: " +str(FN))
print("FP: " + str(FP))
total = TP+FP+FN+TN
print("Total: "+ str(total))
accuracy = (TP+TN)/total
print("Accuracy: " + str(accuracy))


Performance of baseline Q2:
TP: 97797
TN: 2727
FN: 2203
FP: 97273
Total: 200000
Accuracy: 0.50262


ANS: Increasing the value of the threshold (with a smaller  denominator of 0.1 and 0.5) seemed to decrease the accuracy of our purchase predictor closer and closer towards 0.5 (which is equivalent to random prediction). Similarly, decreasing the threshold (with a larger denominator of 3, 10, 200,10000) seemed to also decrease the accuaracy of our predictor closer and closer to 0.5.

With our validation set being biased with a perfect even split of positives (100k) and negative entries(100k), a threshold = 0.5 seems to be the best benchmark for the model. This is because at a Threshold = 0.5 the model is not skewed to have too many FP (which are created if we reduced the threshold below 0.5 because it classifies too many as Positive), and not skewed to have too many FN ( which are created if we increases the threshold above 0.5 because its classified too many as negative). In the end with our 100k non-purchased items being randomly selected and our positives and negative entries being equal in the validation set, it makes sense that we want to have a threshold which parts the training data popularity evenly to best fit the validation data. 

3.) TASK: As users tend to repeatedly purchase items of the same type, build a baseline that returns 'True' if a user has purchased an item of the same category before or zero otherwise

In [4]:
# make list of categories every user has bought
userCatg = dict()
for i in trnData:  # allData: 
    userCatg[i['reviewerID']] = list()
    
# Makes list of categories each user has purchased
for i in trnData:      
    userCatg[i['reviewerID']].append(i['categoryID'])

itemCatg = dict()
# Assigns category to each item
for i in trnData: #allData: 
      itemCatg[i['itemID']] = i['categoryID']

# write predictions into file
predictions = open("predictions_Purchase.txt", 'w')
for l in open("pairs_purchase.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    
    # check if user has bough category and item category has been seen before
    if((u in userCatg.keys()) & (i in itemCatg.keys())):
        uCat = userCatg[u]
        iCat = itemCatg[i]
        if iCat in uCat:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n") 
    else:
        predictions.write(u + '-' + i + ",0\n") # predict as false if user or item has not been seen before        
predictions.close()
    

Q4.) TASK: Upload Solution to and baseline from Q3 to Kaggle

ANS: My Kaggle username is: nickcarmont   
     Score acheived with trnData: 59.7000  
     Max score with all data: 0.6000

Q5.) TASK: Build training/validation sets with only reviews that have a 'categoryID' field. These are reviews that had exactly one of the 5 categories selected, so that there is no ambiguity in their category label (the test reviews also have only a single, unambigious category).

A trivial predictor might assume that each user tends to purchase only one category of item. For each user in the training set, identify  the category of item they purchased most freq (in case of tie, choose which categ is most popular overall). Then, for each revirew in your validation set, simply return whicever category you identified as being the most popular (otherwise return 0 if you've never seen the user before). Report the performance of this classifier on the validation set. 

In [5]:
allDataWCateg = list()

# Filter all data for only data with 'categoryID'
for a in allData:       
    if 'categoryID' in a.keys():
        allDataWCateg.append(a)
        
# part data equally amongst train and val (100k each)      
lenDataCateg = int(len(allDataWCateg)/2)
trnData2 = allDataWCateg[:lenDataCateg]  
valData = allDataWCateg[lenDataCateg:]


# sub-sample data for testing
# trnData2 = trnData2[:20000]  # part data equally amongst train and val
# valData = allDataWCateg[180000:]
# print(len(trnData2))
# print(len(valData))

# most popular categories
categOverallQ5 = {0:0,1:0,2:0,3:0,4:0}
for a in trnData2:
    categOverallQ5[a['categoryID']]+=1
print("most popular categories overall: " + str(categOverallQ5))

# most frequent category by user
userCatgMostF = dict()
for i in trnData2:
    userCatgMostF[i['reviewerID']] = {0:0,1:0,2:0,3:0,4:0}
for i in trnData2:
    userCatgMostF[i['reviewerID']][i['categoryID']] += 1
    
# List of users in trn data without duplicates
users5 = list(set(a['reviewerID'] for a in trnData2))


preds = list()

for a in valData:
    
    # check if user has been seen before in trn data
    if(a['reviewerID'] in users5):
        
        # Get index of most popular category of user
        indMaxVal = max(userCatgMostF[a['reviewerID']], key=userCatgMostF[a['reviewerID']].get)
       
        # in case of tie, return most popular categ overall
        equalEntries = list()
        equalEntries = []
        for i in range(0,5):   # Check for no ties, if ties add to equalEntries
            if((userCatgMostF[a['reviewerID']][indMaxVal]) == (userCatgMostF[a['reviewerID']][i])):
                equalEntries.append(i)
                
        # if equal entries, find max overall
        if(len(equalEntries)>1):
            maxOver = 0
            winCateg = 0
            for j in equalEntries:
                if (maxOver <= categOverallQ5[j]):
                    maxOver = categOverallQ5[j]
                    winCateg = j
                    indMaxVal = winCateg   # set indMaxVal to most popular overall in tie
        # Append max to predictions
        preds.append(indMaxVal)
    else:
        preds.append(0)
        
wrong = 0
corr = 0

# Compare predictions to actual 'CategoryID'
for i in range(0,len(valData)):
    if (valData[i]['categoryID'] == preds[i]):
        corr+=1
    else:
        wrong+=1

print("correct: " +str(corr))
print("wrong: " + str(wrong))
total = corr + wrong
print("total: "  +str(total))
acc = corr/total
print("accuracy: "+ str(acc))

most popular categories overall: {0: 70878, 1: 25512, 2: 1178, 3: 925, 4: 1507}
correct: 80174
wrong: 19826
total: 100000
accuracy: 0.80174


Q6.) TASK: We will now look for the presence of common words. We start by removing punctuation and capitalization and finding the 500 most common words across all reviews ('reviewText' field) in the training set. See the 'text mining' lectures for the code for this process

The most common words probably don't do much help for us to predict categories. Instead, we'll find which words are more common in each category compared to the others. First, we'll compute the frequency of the 500 words you just found as follows:

freq_word = (# times words appears)/(sum of # of items the i_th most popular word appears)

Next, we'll compute the freq per category,e.g:

freq[Women]_word = (# of times words appears in 'Women's clothing' reviews)/ (sum of times the i_th most popular word appears in 'W's Clothing' reviews)

With this computed, we can find which words are more popular in one category compared to their overall frequenyc across categories.i.e

freq['Women's clothing']_word - freq_word

Report for each of the 5 categories the 10 words that are more frequent in that category compared to the other categories.

In [6]:
import string

punct = set(string.punctuation)
wordC = defaultdict(int)

trnData = allData[:100000]
# Remove punctuation and convert to lower case
for a in trnData:  # classify with training data
    for wrd in a['reviewText'].split():
        if(wrd == ''):
            continue
        else:
            wrd = ''.join([char for char in wrd.lower() if not char in punct])
            wordC[wrd] += 1
# print (len(wordC))

wordCRm = wordC
top500 = dict()

while (len(top500)!= 500):

   # Find max in keys
    maxFWord = max(wordCRm, key=wordCRm.get)    
    cWord = wordCRm[maxFWord]
    
    # Add max to new list
    top500.update({maxFWord : cWord})

    # Remove max from old list
    wordCRm.pop(maxFWord)

# Calculate sum # of times the ith most pop word appears
totalAppTop500 = sum(list(top500.values()))

# Make list of frequencies of top 500 words
fTop500 = dict()
for a in top500:
    freqWord = (top500[a])/(totalAppTop500)
    fTop500.update({a:freqWord})
    
# Compute relative frequency of words by category

# find number of times word appears in Categ
punct = set(string.punctuation)
wordCat0 = defaultdict(int)
wordCat1 = defaultdict(int)
wordCat2 = defaultdict(int)
wordCat3 = defaultdict(int)
wordCat4 = defaultdict(int)

totalWC0 =0
totalWC1 =0
totalWC2 =0
totalWC3 =0
totalWC4 =0

trnData = allData[:100000]

# Find top 500 words freq in each category  
# [ SORRY had to do this the long was as I had errors the other way]
for a in trnData:  # classify with training data
    for wrd in a['reviewText'].split():
        if(a['categoryID']==0):
            if(wrd != ''):
                wrd = ''.join([char for char in wrd.lower() if not char in punct])
                if(wrd in (list(fTop500.keys()))):
                    wordCat0[wrd] += 1
                    totalWC0 +=1
        elif(a['categoryID']==1):
            if(wrd != ''):
                wrd = ''.join([char for char in wrd.lower() if not char in punct])
                if(wrd in (list(fTop500.keys()))):
                    wordCat1[wrd] += 1
                    totalWC1 +=1
        elif(a['categoryID']==2):
            if(wrd != ''):
                wrd = ''.join([char for char in wrd.lower() if not char in punct])
                if(wrd in (list(fTop500.keys()))):
                    wordCat2[wrd] += 1
                    totalWC2 +=1
        elif(a['categoryID']==3):
            if(wrd != ''):
                wrd = ''.join([char for char in wrd.lower() if not char in punct])
                if(wrd in (list(fTop500.keys()))):
                    wordCat3[wrd] += 1
                    totalWC3 +=1
        elif(a['categoryID']==4):
            if(wrd != ''):
                wrd = ''.join([char for char in wrd.lower() if not char in punct])
                if(wrd in (list(fTop500.keys()))):
                    wordCat4[wrd] += 1
                    totalWC4 +=1
                    
# Find comparative frequency of top500 words in each category            
for a in list(fTop500.keys()):
    total0 = (wordCat0[a]/totalWC0) - fTop500[a]
    wordCat0.update({a:total0 })
    
    total1 = (wordCat1[a]/totalWC1) - fTop500[a]
    wordCat1.update({a:total1 })
    
    total2 = (wordCat2[a]/totalWC2) - fTop500[a]
    wordCat2.update({a:total2 })
    
    total3 = (wordCat3[a]/totalWC3) - fTop500[a]
    wordCat3.update({a:total3 })
    
    total4 = (wordCat4[a]/totalWC4) - fTop500[a]
    wordCat4.update({a:total4})

w10Cat0 = list()
w10Cat1 = list()
w10Cat2 = list()
w10Cat3 = list()
w10Cat4 = list()
    
# Find 10 words more frequent in every category compared to all others
while(len(w10Cat0)!=10):
    for a in list(fTop500.keys()):
        if((len(w10Cat0)<10)&((wordCat0[a]) > (wordCat1[a])) & ((wordCat0[a]) > (wordCat2[a])) & ((wordCat0[a]) > (wordCat3[a])) & ((wordCat0[a]) > (wordCat4[a]))):
              w10Cat0.append(a)

while(len(w10Cat1)!=10):
    for a in list(fTop500.keys()):
        if((len(w10Cat1)<10)&((wordCat1[a]) > (wordCat0[a])) & ((wordCat1[a]) > (wordCat2[a])) & ((wordCat1[a]) > (wordCat3[a])) & ((wordCat1[a]) > (wordCat4[a]))):
              w10Cat1.append(a)
                
while(len(w10Cat2)!=10):
    for a in list(fTop500.keys()):
        if((len(w10Cat2)<10)&((wordCat2[a]) > (wordCat1[a])) & ((wordCat2[a]) > (wordCat0[a])) & ((wordCat2[a]) > (wordCat3[a])) & ((wordCat2[a]) > (wordCat4[a]))):
              w10Cat2.append(a)

while(len(w10Cat3)!=10):
    for a in list(fTop500.keys()):
        if((len(w10Cat3)<10)&((wordCat3[a]) > (wordCat1[a])) & ((wordCat3[a]) > (wordCat2[a])) & ((wordCat3[a]) > (wordCat0[a])) & ((wordCat3[a]) > (wordCat4[a]))):
              w10Cat3.append(a)

while(len(w10Cat4)!=10):
    for a in list(fTop500.keys()):
        if((len(w10Cat4)<10)&((wordCat4[a]) > (wordCat1[a])) & ((wordCat4[a]) > (wordCat2[a])) & ((wordCat4[a]) > (wordCat3[a])) & ((wordCat4[a]) > (wordCat0[a]))):
              w10Cat4.append(a)
                
print("10 words more frequent in each category than the rest:")            
print("Women (category 0) : " +str(w10Cat0))
print("Men (category 1): " +str(w10Cat1))
print("Girls (category  2): " +str(w10Cat2))
print("Boys (category 3): " +str(w10Cat3))
print("Baby (category 4): " +str(w10Cat4))


10 words more frequent in each category than the rest:
Women (category 0) : ['i', 'in', 'but', 'have', 'not', 'with', 'like', 'wear', 'love', 'comfortable']
Men (category 1): ['the', 'a', 'to', 'of', 'that', 'you', 'good', 'or', 'if', 'more']
Girls (category  2): ['it', 'is', 'this', 'be', 'would', 'its', 'one', 'price', 'small', 'looks']
Boys (category 3): ['for', 'my', 'was', 'size', 'fit', 'great', '', 'well', 'up', 'nice']
Baby (category 4): ['and', 'they', 'are', 'these', 'on', 'them', 'so', 'very', 'as', 'just']


Q7.) TASK: Now we'll run some classifiers by  building a simple feature vector out of our 500 most popular words as follows:
[commonWords[0] in review, commonWords[1] in review,..., commonWords[499] in review]

We will train an SVM to distinguish Women's clothing and non-women's clothing only. That is, discard all other categories from the training set in order to train a binary classifier, but keep them in the validation set ( your classifier will simply be wrong for those instances). Train for C (= 0.01, 0.1, 1,10,100), the regularisation parameter for the SVM, and report the best performance you obtain on  the validation set.

In [ ]:
import numpy
import urllib
import scipy.optimize
import random
from sklearn import svm
from sklearn.metrics import accuracy_score

trnData = allData[:100000]
# Check if categ and review values are in trnData
trnData = [x for x in trnData if 'reviewText' in x if 'categoryID' in x]

# Change C-value and find highest accuracy on val Data
cVal = 100

# Sub-sample training data to 10% for testing
random.shuffle(trnData)
trnData  = trnData[90000:]
print("length Training Data: " +str(len(trnData)))


# Sub-sample validation data to 10% for testing
random.shuffle(validationData)
validationData  = validationData[190000:]
print("length Validation Data: "+str(len(validationData)))

def isWomenClothing(dataIn):
    if dataIn['categoryID'] == 0:
        isW = 1 
    else:
        isW = 0 
    return isW

def isWordInReview(dataIn, word):
    if word in dataIn:
        return 1 
    else:
        return 0 

def features(dataIn):
    feat = list()
    for a in list(fTop500.keys()):
         feat.append(isWordInReview(dataIn['reviewText'],a))                                                                                 
    return feat

# Test Data in features
xClass= [features(b) for b in trnData]
yClass= [isWomenClothing(w) for w in trnData]

# SVM linear Classifier
wClassifier = svm.SVC(C=cVal, kernel='linear')
wClassifier.fit(xClass, yClass)

# Validation data
xVal= [features(b) for b in validationData]
yVal= [w['categoryID'] for w in validationData]

predClassTst = wClassifier.predict(xVal)
print("accuracy for C= " +str(cVal)+" : "+str(accuracy_score(yVal, predClassTst)))

length Training Data: 10000
length Validation Data: 10000


ANS: As commented by the TA's I ran my model using the different C values of 0.01,0.1,1,10,100 on the 10% sampled data. As the SVM was taking too long to run on the full data set.   
  
Given this, I found that a C value (regularization paramter) of 0.1 lead to the highest accuracy for the SVM of Acc = 0.8174.  

Q8.) TASK: Rather than training a binary SVM, train 5 SVM's that distinguish each category from all other categories. Again selecting different values for C (= 0.01, 0.1, 1, 10, 100) for each classifier. For the validation set, classify each point according to whichever of the five classifiers is most confident about the label being positive (use 'decision_function' in  the SVM lib). Report the performance of this classifier on the validation set and upload the solution to Kaggle by running it on the test data.

In [ ]:
# Make functions distinguishing category y data for each SVM

def isMenClothing(dataIn):
    if dataIn['categoryID'] == 1:
        isC = 1 
    else:
        isC = 0 
    return isC

def isGirlClothing(dataIn):
    if dataIn['categoryID'] == 2:
        isC = 1 
    else:
        isC = 0 
    return isC

def isBoyClothing(dataIn):
    if dataIn['categoryID'] == 3:
        isC = 1 
    else:
        isC = 0 
    return isC

def isBabyClothing(dataIn):
    if dataIn['categoryID'] == 4:
        isC = 1 
    else:
        isC = 0 
    return isC


# SVM 1 for Women's Clothing: 

cVal = 0.01
xClass= [features(b) for b in trnData]
yClass= [isWomenClothing(w) for w in trnData]

# SVM linear Classifier
wClassifier = svm.SVC(C=cVal, kernel='linear')
wClassifier.fit(xClass, yClass)

# Validation data
xVal= [features(b) for b in validationData]
yVal= [w['categoryID'] for w in validationData]

predClassTst = wClassifier.predict(xVal)
print("accuracy for C= " +str(cVal)+" : "+str(accuracy_score(yVal, predClassTst)))


# SVM 2 for Mens's Clothing:

cVal = 100
yClass= [isMenClothing(w) for w in trnData]

# SVM linear Classifier
mClassifier = svm.SVC(C=cVal, kernel='linear')
mClassifier.fit(xClass, yClass)

# Validation data
xVal= [features(b) for b in validationData]
yVal= [w['categoryID'] for w in validationData]

predClassTst = mClassifier.predict(xVal)
print("accuracy for C= " +str(cVal)+" : "+str(accuracy_score(yVal, predClassTst)))


# SVM 3 for Girls's Clothing:

cVal = 0.1
yClass= [isGirlClothing(w) for w in trnData]

# SVM linear Classifier
gClassifier = svm.SVC(C=cVal, kernel='linear')
gClassifier.fit(xClass, yClass)

# Validation data
xVal= [features(b) for b in validationData]
yVal= [w['categoryID'] for w in validationData]

predClassTst = gClassifier.predict(xVal)
print("accuracy for C= " +str(cVal)+" : "+str(accuracy_score(yVal, predClassTst)))


# SVM 4 for Boys's Clothing:

cVal = 10
yClass= [isBoyClothing(w) for w in trnData]

# SVM linear Classifier
bClassifier = svm.SVC(C=cVal, kernel='linear')
bClassifier.fit(xClass, yClass)

# Validation data
xVal= [features(b) for b in validationData]
yVal= [w['categoryID'] for w in validationData]

predClassTst = bClassifier.predict(xVal)
print("accuracy for C= " +str(cVal)+" : "+str(accuracy_score(yVal, predClassTst)))


# SVM 5 for Boys's Clothing:

cVal = 0.01
yClass= [isBabyClothing(w) for w in trnData]

# SVM linear Classifier
bbClassifier = svm.SVC(C=cVal, kernel='linear')
bbClassifier.fit(xClass, yClass)

# Validation data
xVal= [features(b) for b in validationData]
yVal= [w['categoryID'] for w in validationData]

predClassTst = bbClassifier.predict(xVal)
print("accuracy for C= " +str(cVal)+" : "+str(accuracy_score(yVal, predClassTst)))

